In [1]:
import numpy as np
import torch
import pandas as pd
import torchvision
from torchvision import datasets,transforms
from torch import nn
from torch.nn import functional as F
from torchsummary import summary

transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                               transforms.Normalize(mean=[0.5,],std=[0.5,])])

data_train = datasets.FashionMNIST(root="./data",
                                   transform=transform,
                                   train=True,
                                   download=True)
data_test = datasets.FashionMNIST(
    root="./data",
    transform=transform,
    train=False
)

data_loader_train = torch.utils.data.DataLoader(dataset=data_train,
                                                batch_size=256,
                                                shuffle=True)
data_loader_test = torch.utils.data.DataLoader(dataset=data_test,
                                               batch_size=256,
                                               shuffle=True)
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,6,5,padding=2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2,2)
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

lr = 0.5
num_epochs = 10
net = Net()
summary(net,(1,28,28))
optimizer = torch.optim.SGD(net.parameters(),lr = lr)
loss_fn = nn.CrossEntropyLoss()
for epoch in range(num_epochs):
    net.train()
    for idx,(train_x,train_label) in enumerate(data_loader_train):
        optimizer.zero_grad()
        pred_y = net(train_x)
        loss = loss_fn(pred_y,train_label.long())
        if idx % 10000 == 0:
            print("loss:{:.4f}".format(loss))
        loss.backward()
        optimizer.step()
    all_correct_num = 0
    all_sample_num = 0
    net.eval()
    for i, (test_x,test_label) in enumerate(data_loader_test):
        pred_y = net(test_x.float()).detach()
        pred_y = np.argmax(pred_y,axis=-1)
        current_correct_num = pred_y == test_label
        all_correct_num += np.sum(current_correct_num.numpy(),axis=-1)
        all_sample_num += current_correct_num.shape[0]
    acc = all_correct_num / all_sample_num
    print("epoch {:d}: acc = {:.4f}".format(epoch,acc))




----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
            Conv2d-2           [-1, 16, 10, 10]           2,416
            Linear-3                  [-1, 120]          48,120
            Linear-4                   [-1, 84]          10,164
            Linear-5                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.24
Estimated Total Size (MB): 0.29
----------------------------------------------------------------


RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x46656 and 400x120)